# **Economic Freedom and Happiness Across Europe**

In [1]:
import pandas as pd
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pycountry 

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Function for converting to iso 
def getIsoCode(country_name):
    try:
        country_iso = pycountry.countries.search_fuzzy(country_name)[0]
        return country_iso.alpha_3
    except LookupError:
        return ""

In [2]:
%%html
<style>
.cartesianlayer .bars path,
.colorbar .cbfills,
.slider-rail-rect { clip-path: inset(0% 0% 0% 0% round 6px); }

.legendundefined { clip-path: inset(0% 0% 0% 0% round 3px); }

.gtitle .line:nth-of-type(2) { fill: grey; }

.output_area:last-of-type,
.cell_output {
    filter: drop-shadow(0px 0px 10px rgba(0, 0, 0, 0.2));
    -webkit-backdrop-filter: blur(0); // fixes a safari drop-shadow bug
}

#notebook-container img,
main img {
    box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.2);
}

#notebook-container img,
main img,
.plotly-graph-div {
    border-radius: 10px;
    overflow: hidden;
    border: 2px solid black;
    border-bottom: 3px solid black;
}
</style>

In [3]:
happiness_data = pd.read_csv("happiness.csv")
happiness_data["Country"] = happiness_data["Country or region"]
happiness_data = happiness_data.drop(columns=["Country or region", "Overall rank"])
happiness_data = happiness_data.rename(columns={"Score": "Overall Happiness"})

In [4]:
freedom_data = pd.read_csv("freedom.csv", low_memory=False)
# Normalize country column
freedom_data = freedom_data.rename(columns={"countries": "Country"})

# Filter out years other than 2019
freedom_data["year"] = pd.to_numeric(freedom_data["year"], errors='coerce')
freedom_data = freedom_data[freedom_data["year"] == 2019]

# Reset and Drop
freedom_data = freedom_data.reset_index()
freedom_data = freedom_data.drop(columns="index")

freedom_filtered = freedom_data[freedom_data['Country'].isin([
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", 
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", 
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", 
    "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", 
    "Liechtenstein", "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", 
    "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", 
    "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", 
    "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Vatican City"
])]

columns_to_keep = [
    'Country', 'ef_government', 'ef_gender', 'ef_legal', 'ef_money', 'ef_trade', 'ef_regulation', 'ef_rank'
]

freedom_filtered = freedom_filtered[columns_to_keep]

In [5]:
data = pd.merge(happiness_data, freedom_filtered, on='Country')
data = data.drop_duplicates(subset=['Country'])

data["iso"] = data['Country'].apply(getIsoCode)

# CREATING NEW RANK
sorted_df = data.sort_values(by='ef_rank')
sorted_df['ef_rank'] = range(1, len(sorted_df) + 1)

# Display the sorted DataFrame with ranks
data = sorted_df

## Analysis Across Countries

In [6]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# Assuming 'data' is your DataFrame containing "ef_rank" and "Overall Happiness" columns

# Create a scatter plot
fig = go.Figure()

# Add scatter plot
fig.add_trace(go.Scatter(
    x=data["ef_rank"],
    y=data["Overall Happiness"],
    mode='markers',  # Scatter plot with markers only
    name='Overall Happiness'
))

# Calculate the trendline using numpy.polyfit
z = np.polyfit(data["ef_rank"], data["Overall Happiness"], 1)
p = np.poly1d(z)

# Add trendline to plot
fig.add_trace(go.Scatter(
    x=data["ef_rank"],
    y=p(data["ef_rank"]),
    mode='lines',  # Line plot for the trendline
    name='Trendline',
    line=dict(color='black', width=2)  # Customize the trendline color and width
))

# Update layout
fig.update_layout(
    width=700,
    height=500,
    title='Overall Happiness vs. Economic Freedom Rank',
    xaxis_title='Economic Freedom Rank',
    yaxis_title='Overall Happiness',
    yaxis_range=[0, 10],
    showlegend=False  # Disable legend
)

# Show the figure
pio.show(fig)


## Top vs. Bottom performers

In [7]:
top_3 = data.nlargest(3, 'ef_rank')
bottom_3 = data.nsmallest(3, 'ef_rank')

## Dimensional Analysis

In [8]:
columns_to_keep=['GDP per capita', 'Social support',
       'Healthy life expectancy', 'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'ef_government', 'ef_gender',
       'ef_legal', 'ef_money', 'ef_trade', 'ef_regulation']

corr_matrix = data[columns_to_keep].corr()

fig_corr_matrix = px.imshow(corr_matrix,
                            title='Correlation Matrix of Freedom, Happiness, and Health Indicators',
                            labels=dict(color='Correlation'),
                            color_continuous_scale='RdBu')

fig_corr_matrix.update_layout(
    width=700,
    height=500,
    font=dict(
        size=10,  # Adjust the font size here
    ),
    coloraxis_colorbar=dict(
        title="Correlation",
        tickvals=[-1, 0, 1],  # Example colorbar tick values
        ticktext=["Negative", "Neutral", "Positive"],  # Example colorbar tick labels
    )
)

fig_corr_matrix.show()